### This code allows to merge LEIE Dataset and Medicare Part D dataset with Selected Columns and Filters

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

# 1- Processing LEIE Dataset

In [ ]:
# Loads the 'LEIE_UPDATED.csv' file into a pandas DataFrame 
leie = pd.read_csv('LEIE_UPDATED.csv')

In [ ]:
# Displays the dimensions
leie.shape

In [ ]:
# Print column names
print("Column Names:")
print(leie.columns.tolist())

In [ ]:
leie.head()

In [ ]:
# Define a list of specific exclusion codes to be used for filtering the dataset (from paper)
exclusion_codes = ['1128a1', '1128a2', '1128a3', '1128b4', '1128b7', '1128c3gi', '1128c3gii']

# Filter the 'leie' DataFrame to include only rows where the 'EXCLTYPE' column's value is present in the 'exclusion_codes' list.
filtered_leie = leie[leie.EXCLTYPE.isin(exclusion_codes)]
filtered_leie.shape

In [ ]:
# Convert the 'EXCLDATE' column to datetime objects using the specified year-month-day format.
filtered_leie['EXCLDATE'] = pd.to_datetime(filtered_leie.EXCLDATE, format='%Y%m%d') #most important value

In [ ]:
def calculate_exclusion_end(exclusion_date, end_year): 
    """
        # Defines a function to calculate the exclusion end year 
        # based on the exclusion date and a given end year.
    """
    month = exclusion_date.month # Extracts the month from the 'exclusion_date'.
    if month > 6: # Checks if the exclusion month is after June (i.e., July to December).
        return end_year + 1 
    else: # If the exclusion month is June or earlier (i.e., January to June).
        return end_year 

In [ ]:
filtered_leie.head()

# 2- Processing Medicare Part D Dataset

In [ ]:
# Loads the 'Combined_Medicare_2017_2019.csv' file into a pandas DataFrame 
partd_main = pd.read_csv('outputs/Combined_Medicare_2017_2019.csv')

In [ ]:
# Displays the dimensions
partd_main.shape

In [ ]:
partd_main.head()

In [ ]:
print("Column Names:")
print(partd_main.columns.tolist())

In [ ]:
# Renames the column 'Prscrbr_NPI' to 'npi' in the 'partd_main' DataFrame, modifying the DataFrame in place.
partd_main.rename({'Prscrbr_NPI':'npi'}, axis=1, inplace=True)

In [16]:
# make NPI lowercase so we can merge on it later. 
filtered_leie.rename({'NPI':'npi'}, axis=1, inplace=True)

In [ ]:
partd_main.head()

In [ ]:
 # Renames the column 'Source_Year' to 'DATA_YEAR' in the 'partd_main' DataFrame, modifying the DataFrame in place.
partd_main.rename(columns={'Source_Year': 'DATA_YEAR'}, inplace=True)

In [ ]:
# Replaces all occurrences of '0' with '5' in the 'Tot_Benes' column of the 'partd_main' DataFrame.
partd_main.Tot_Benes = partd_main.Tot_Benes.replace(0,5)

In [ ]:
# Group the 'partd_main' DataFrame by 'npi', 'Prscrbr_Type', and 'DATA_YEAR' columns.
# For each group, compute multiple aggregation statistics (sum, mean, median, std, min, max)
# on the following numeric columns: 'Tot_Benes', 'Tot_Clms', 'Tot_30day_Fills',
# 'Tot_Day_Suply', and 'Tot_Drug_Cst'. Then print the resulting DataFrame's shape.


partd_main = partd_main.groupby(['npi','Prscrbr_Type', 'DATA_YEAR']).agg({'Tot_Benes':['sum', 'mean', 'median', np.std, 'min', 'max'],
                                    'Tot_Clms':['sum', 'mean', 'median', np.std, 'min', 'max'],
                                    'Tot_30day_Fills':['sum', 'mean', 'median', np.std, 'min', 'max'],
                                    'Tot_Day_Suply':['sum', 'mean', 'median', np.std, 'min', 'max'],
                                    'Tot_Drug_Cst': ['sum', 'mean', 'median', np.std, 'min', 'max']})
print(partd_main.shape)

In [ ]:
# Flatten the MultiIndex columns resulting from the aggregation by joining each level with an underscore.
# For example, ('Tot_Benes', 'sum') becomes 'Tot_Benes_sum'.

partd_main.columns = ['_'.join(col) for col in partd_main.columns.values]

In [ ]:
partd_main.columns

In [ ]:
# Reset the index for the specified grouping columns ('Prscrbr_Type', 'npi', 'DATA_YEAR'),
# moving them back into regular columns in the DataFrame for easier access and analysis.

partd_main = partd_main.reset_index(level=['Prscrbr_Type', 'npi', 'DATA_YEAR'])

In [ ]:
partd_main.head()

# 3- Merging Medicare Part D and LEIE 

In [ ]:
# Merge the 'filtered_leie' and 'partd_main' DataFrames on the 'npi' column using an outer join.
# This ensures that all records from both DataFrames are retained, with NaNs where data is missing.


partd = pd.merge(filtered_leie, partd_main, on='npi', how='outer')

In [ ]:
partd.columns

In [ ]:
# Reorder and subset the 'partd' DataFrame to include only the relevant columns:
# - Identifier and grouping columns: 'npi', 'Prscrbr_Type', 'DATA_YEAR'
# - Aggregated statistics for prescription-related metrics (e.g., beneficiaries, claims, cost)
# - LEIE exclusion-related columns: 'EXCLTYPE', 'EXCLDATE', 'WVRSTATE'
# This helps streamline the dataset for downstream analysis or modeling.


partd = partd[[
        'npi', 'Prscrbr_Type', 'DATA_YEAR',       
       'Tot_Benes_sum', 'Tot_Benes_mean', 'Tot_Benes_median',
       'Tot_Benes_std', 'Tot_Benes_min', 'Tot_Benes_max',

       'Tot_Clms_sum', 'Tot_Clms_mean',
       'Tot_Clms_median', 'Tot_Clms_std',
       'Tot_Clms_min', 'Tot_Clms_max',

       'Tot_30day_Fills_sum', 'Tot_30day_Fills_mean',
       'Tot_30day_Fills_median', 'Tot_30day_Fills_std',
       'Tot_30day_Fills_min', 'Tot_30day_Fills_max',

       'Tot_Day_Suply_sum', 'Tot_Day_Suply_mean',
       'Tot_Day_Suply_median', 'Tot_Day_Suply_std',
       'Tot_Day_Suply_min', 'Tot_Day_Suply_max',
       
       'Tot_Drug_Cst_sum','Tot_Drug_Cst_mean', 
       'Tot_Drug_Cst_median', 'Tot_Drug_Cst_std',
       'Tot_Drug_Cst_min', 'Tot_Drug_Cst_max', 
       
       'EXCLTYPE',
       'EXCLDATE', 'WVRSTATE' ]]

### Remove rows without npi or provider

In [ ]:
# Filter out rows where 'npi' is 0 (invalid identifier) or 'Prscrbr_Type' is missing/empty.
# This ensures that only valid, non-null prescriber records are retained in the DataFrame.


partd = partd[(partd.npi != 0) & (partd.Prscrbr_Type)]
partd.shape

### Add target (label)

In [ ]:
# Add a new column 'TARGET' to the 'partd' DataFrame and set its default value to '0' (as a string).
# This will likely be used later for labeling or classification purposes (e.g., marking non-excluded providers).


partd['TARGET'] = '0'

In [ ]:
# Extract the year from the 'EXCLDATE' datetime column and store it in a new column 'START_EXCLDATE'.
# This helps in analyzing or grouping exclusions by year.


partd['START_EXCLDATE'] = partd['EXCLDATE'].dt.year

In [ ]:
partd.head()

In [ ]:
def make_labels(start_exc, data_yr):
    """
    Assigns a fraud label based on the relationship between exclusion year and data year.

    Parameters:
    - start_exc (int or float): The year the provider was excluded (e.g., from EXCLDATE).
    - data_yr (int): The year of the Part D dataset record.

    Returns:
    - str: 'FRAUD' if the exclusion happened after the data year (i.e., provider was still active),
           'NOT_FRAUD' otherwise (i.e., already excluded or clean).
    """
    if start_exc > data_yr:
        return 'FRAUD'
    else:
        return 'NOT_FRAUD'


In [ ]:
# Create the 'TARGET' column by applying the 'make_labels' function row-wise,
# comparing 'START_EXCLDATE' and 'DATA_YEAR' to assign fraud labels for each record.

partd['TARGET'] = partd[["START_EXCLDATE","DATA_YEAR"]].apply(lambda x: make_labels(*x), axis=1)

In [ ]:
# Get the list of unique label values present in the 'TARGET' column,
# ordered by their frequency from most to least common.


partd.TARGET.value_counts().index.to_list()

In [ ]:
partd.TARGET.value_counts()

# TARGET
# NOT_FRAUD    2854225
# FRAUD           2614
# Name: count, dtype: int64

# 4- Downsizing Dataset Randomly 

In [ ]:
# Total samples desired
desired_total = 1_000_000

# Calculate number of samples per class based on the original ratio
fraud_ratio = partd["TARGET"].value_counts(normalize=True)["FRAUD"]
not_fraud_ratio = partd["TARGET"].value_counts(normalize=True)["NOT_FRAUD"]

num_fraud = int(desired_total * fraud_ratio)
num_not_fraud = desired_total - num_fraud  # or int(desired_total * not_fraud_ratio)

print(f"Sampling {num_fraud} FRAUD and {num_not_fraud} NOT_FRAUD records.")

# Separate classes
df_fraud = partd[partd["TARGET"] == "FRAUD"]
df_not_fraud = partd[partd["TARGET"] == "NOT_FRAUD"]

# Sample with replacement = False (downsampling)
df_fraud_sampled = df_fraud.sample(n=num_fraud, random_state=42)
df_not_fraud_sampled = df_not_fraud.sample(n=num_not_fraud, random_state=42)

# Combine back
df_sampled = pd.concat([df_fraud_sampled, df_not_fraud_sampled])

# Shuffle the resulting dataframe (optional but recommended)
df_sampled = df_sampled.sample(frac=1, random_state=42).reset_index(drop=True)

# Check new class distribution
print("New TARGET distribution:")
print(df_sampled["TARGET"].value_counts())
print(df_sampled["TARGET"].value_counts(normalize=True))
print(f"New dataset size: {len(df_sampled)}")


# 5- One Hot Encoding for Categorical Columns

In [38]:
partd_category_columns = ['Prscrbr_Type']

In [ ]:
# Convert categorical columns listed in 'partd_category_columns' into one-hot encoded dummy variables,
# dropping the first category in each to avoid multicollinearity in modeling.

df_sampled = pd.get_dummies(df_sampled, columns=partd_category_columns, drop_first=True)


In [ ]:
# Specify columns to drop from the dataset, typically because they are identifiers,
# date-related, or exclusion info not needed for model training or further analysis.


columns_to_drop = [ 'EXCLTYPE','EXCLDATE',
                   'WVRSTATE',
                   'START_EXCLDATE', 'npi', 'DATA_YEAR']
df_sampled.drop(columns_to_drop, axis=1, inplace=True)

In [ ]:
df_sampled.shape

In [ ]:
# Replace all missing (NaN) values in 'df_sampled' with 0 to avoid issues in modeling or analysis,
# then display the first few rows to verify the changes.

df_sampled.fillna(0, inplace=True)
df_sampled.head()

In [ ]:
df_sampled.columns.tolist()

In [ ]:
# Save the processed DataFrame 'df_sampled' to a CSV file without the index column,
# creating a downsized dataset for future use or sharing.


df_sampled.to_csv("Combined_LEIE_Medicare_2017_2019_DOWNSIZED_1mil.csv", index=False)